# Homework 1 (2022) : Data Wrangling and Visualization

Due date : **2022-01-30 @23h55** (this is a **hard deadline**)

## Fill this cell with your names

- Ali, Shaïna, Informatique/Mathématique-Informatique
- Apanishile, Tolulope, Informatique/Mathématique-Informatique

## Carefully follow instructions given on the [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/)

Write in English or French

- The deliverable is a `xxx.ipynb` file (jupyter notebook) or a `xxx.py` file (if you are using `jupytext`) built by completing the template. **We won't execute the code in your notebook:** all your results, displays and plots must be visible without having to rerun everything.

Once again, follow the steps described here : [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/). **If you don't: no evaluation!**

## Grading <i class="fa graduation-cap"></i>

Here is the way we'll assess your work

| Criterion | Points | Details |
|:----------|:-------:|:----|
|Spelling and syntax | 3 | English/French  |
|Plots correction | 3 |  Clarity / answers the question  |
|Plot style and cleanliness | 3 | Titles, legends, labels, breaks ... |
|Table wrangling | 4 | ETL, SQL like manipulations |
|Computing Statistics | 5 | SQL `goup by`  and aggregation  |
|DRY compliance | 2 | DRY principle at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)|

If we see a single (or more) `for` loop in your code: **-5 points**.  Everything can be done using high-level `pandas` methods

# Preliminaries

## Notebook: Modus operandi

- This is a [Jupyter Notebook]().
- When you execute code within the notebook, the results appear beneath the code.
- [Jupytext]()

## Packages

- Base `Python` can do a lot. But the full power of `Python` comes from a fast growing collection of `packages`/`modules`.

- Packages are first installed (that is using `pip install` or `conda install`), and if
needed, imported during a session.

- The `docker` image you are supposed to use already offers a lot of packages. You should not need to install new packages.

- Once a package has been installed on your drive, if you want all objects exported by the package to be available in your session, you should import the package, using `from pkg import *`.

- If you just want to pick some sobjects from the package,
you can use qualified names like `pkg.object_name` to access the object (function, dataset, class...)


In [1]:
# importing basic tools
import numpy as np
import pandas as pd

from pandas.api.types import CategoricalDtype

import os            # file operations
import requests      # networking

from datetime import date  # if needed

In [2]:
# importing plotting packages
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [3]:
# make pandas plotly-friendly
np.set_printoptions(precision=2, suppress=True)
%matplotlib inline
pd.options.plotting.backend = "plotly"

# Getting the data

The data are built and made available by [INSEE](https://www.insee.fr/fr/accueil)  (French Governement Statistics Institute)

Prénoms:
- [https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip](https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip)

This dataset has been growing for a while. It has been considered by
social scientists for decades.  Given names are meant to give insights into a variety
of phenomena, including religious observance.

- A glimpse at the body of work can be found in [_L'archipel français_ by Jérome Fourquet, Le Seuil, 2019 ](https://www.seuil.com/ouvrage/l-archipel-francais-jerome-fourquet/9782021406023)

- Read the [File documentation](https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262#documentation)

## Download the data

**QUESTION:** Download the data into a file which relative path is `'./nat2020_csv.zip'`

__Hints:__

- Have a look at [`requests`](https://requests.readthedocs.io/en/master/).
- Use magic commands to navigate across the file hierarchy and create subdirectories
when needed

In [4]:
params = dict(
    url = 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip',
    dirpath = './',
    timecourse = '',
    datafile = 'nat2020.hdf',
    fpath = 'nat2020_csv.zip'
)

In [5]:
# modify location  make sure you are in the right directory
# %cd
# %pwd  #
# %ls
# %mkdir # if needed

In [6]:
url = params['url']      # 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip'
fpath = params['fpath']  # './nat2020_csv.zip'

In [7]:
# your code here
dirpath = params['dirpath']

if os.path.exists(os.path.join(dirpath, fpath)):
    print('The file %s already exists.' % os.path.join(dirpath, fpath))
else:
    r = requests.get(url)
    with open(os.path.join(dirpath, fpath), 'wb') as f:
        f.write(r.content)
    print('Downloaded file %s.' % os.path.join(dirpath, fpath))

#shall I unzip the file now? Apparently unnecessary
"""from zipfile import ZipFile
zf = ZipFile('./nat2020_csv.zip', 'r')
zf.extractall('./')
zf.close()"""


The file ./nat2020_csv.zip already exists.


"from zipfile import ZipFile\nzf = ZipFile('./nat2020_csv.zip', 'r')\nzf.extractall('./')\nzf.close()"

## Load the data in memory

**QUESTION:** Load the data in a `pandas` `DataFrame` called `data`

__Hints:__

- You should obtain a `Pandas dataframe` with 4 columns.
- Mind the conventions used to build theis `csv` file.
- Package `pandas` provides the convenient tools.
- The dataset, though not too large, is already demanding.
- Don't hesitate to test your methods on a sample of rows method `sample()` from class `DataFrame` can be helpful.

In [8]:
# your code here
data = pd.read_csv("nat2020_csv.zip", sep = ";")
data.sample(n=10)

,sexe,preusuel,annais,nombre
464673,2,JUANA,1957,10
379830,2,DANIELE,1968,442
430908,2,GUÉNOLA,1979,5
258644,1,SHAMS,2020,13
475417,2,KENZA,1993,95
121139,1,HUBERT,1920,633
140958,1,JEAN-PAUL,2017,9
222394,1,OREN,2011,28
197948,1,MEHMET,2020,65
659440,2,YASSMINA,1990,4


## Explore the data

**QUESTION:** Look at the data, Use the attributes `columns`, `dtypes` and the methods `head`, `describe`, to get a feeling of the data.

- This dataset is supposed to report all given names used
for either sex during a year in France since 1900

- The file is made of `652 056` lines and  4 columns.

```
|-- preusuel : object
|-- nombre: int64
|-- sexe: int64
|-- annais: object
```

Each row indicates for a given `preusuel` (prénom usuel, given name), `sexe` (sex), and `annais` (année naissance, birthyear) the `nombre` (number) of babies of the given sex who were given that name during the given year.

|sexe    |preusuel     | annais|   nombre|
|:------|:--------|----:|---:|
|2     |SYLVETTE | 1953| 577|
|1   |BOUBOU   | 1979|   4|
|1   |NILS     | 1959|   3|
|2   |NICOLE   | 2003|  36|
|1   |JOSÉLITO | 2013|   4|


**QUESTION:** Compare memory usage and disk space used by data

**Hints:**

- The method `info`  prints a concise summary of a `DataFrame`.
- With optional parameter `memory_usage`, you can get an estimate
of the amount of memory used by the `DataFrame`.
- Beware that the resulting estimate depends on the argument fed.

In [9]:
# your code here

data.dtypes


sexe         int64
preusuel    object
annais      object
nombre       int64
dtype: object

In [10]:
data.columns

Index(['sexe', 'preusuel', 'annais', 'nombre'], dtype='object')

In [11]:
data.head(15)

,sexe,preusuel,annais,nombre
0,1,_PRENOMS_RARES,1900,1250
1,1,_PRENOMS_RARES,1901,1342
2,1,_PRENOMS_RARES,1902,1330
3,1,_PRENOMS_RARES,1903,1286
4,1,_PRENOMS_RARES,1904,1430
5,1,_PRENOMS_RARES,1905,1472
6,1,_PRENOMS_RARES,1906,1451
7,1,_PRENOMS_RARES,1907,1514
8,1,_PRENOMS_RARES,1908,1509
9,1,_PRENOMS_RARES,1909,1526


**QUESTION:** Display the output of `.describe()` with style.

In [12]:


# your code here
(
    
data
    .describe()
    .style
    .set_properties(**{'background-color': 'white',                                                                         
                                    'border-color': 'white',
                                    'border': '1.3px',
                                    'color': 'blue'})
    #.format("{:.3}")
    .set_precision(2)
)
 

,sexe,nombre
count,667364.00,667364.00
mean,1.54,129.77
std,0.50,884.91
min,1.00,1.00
25%,1.00,4.00
50%,2.00,8.00
75%,2.00,25.00
max,2.00,53584.00


**QUESTION:** For each column compute the number of distinct values

In [13]:
# your code here
data.nunique()

sexe            2
preusuel    35010
annais        122
nombre       7304
dtype: int64

# Transformations

## Improving the data types

**QUESTION:** Make `sexe` a category with two levels `Female` and `Male`. Call the new column `genre`. Do you see any reason why this factor should be ordered?

__Hint:__ Read [Pandas and categorical variables](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=category)

In [14]:

data['genre'] = data['sexe'].astype('category').cat.rename_categories({1:'Male', 2:'Female'})



In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667364 entries, 0 to 667363
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   sexe      667364 non-null  int64   
 1   preusuel  667362 non-null  object  
 2   annais    667364 non-null  object  
 3   nombre    667364 non-null  int64   
 4   genre     667364 non-null  category
dtypes: category(1), int64(2), object(2)
memory usage: 21.0+ MB


In [16]:
data.sample(10)

,sexe,preusuel,annais,nombre,genre
100724,1,GAYE,1983,5,Male
333385,2,ANNETTE,1907,230,Female
605437,2,ROSALIND,XXXX,34,Female
251638,1,SALLAH,1956,3,Male
363349,2,CHADIA,1987,25,Female
377514,2,CYRIANE,2002,29,Female
126173,1,INSA,XXXX,29,Male
461044,2,JOCELYNE,1974,253,Female
451351,2,ISAURA,2017,8,Female
213259,1,NATHIS,XXXX,5,Male


**QUESTION:** Compare memory usage of columns `sexe` and `genre`

In [17]:
# your code here
data.memory_usage()


Index           128
sexe        5338912
preusuel    5338912
annais      5338912
nombre      5338912
genre        667488
dtype: int64

**QUESTION:** Would it be more memory-efficient to recode `sexe` using modalities `F` and `M` instead of `Male` and `Female` ?

_Insert your answer here_

> It would not be more effecient. The same amount of memory is used for all categorical types 

## Dealing with missing values

**QUESTION:** Variable `annais` class is `object`. Make `annais` of type `float`. Note that missing years are encoded as "XXXX", find a way to deal with that.

__Hint:__  As of releasing this Homework (2021-01-22), `Pandas` is not very good at managing missing values,
see [roadmap](https://pandas.pydata.org/docs/development/roadmap.html). Don't try to convert `annais` into an integer column.

In [18]:
# your code here
data['annais']=pd.to_numeric(data['annais'],errors='coerce').astype(float)

In [19]:
data.dtypes

sexe           int64
preusuel      object
annais       float64
nombre         int64
genre       category
dtype: object

## Rename and remove columns

**QUESTION:** Remove useless columns (now that you've created new ones, and rename them). You should end up with a dataframe with columns called `"gender"`, `"year"`, `"count"`, `"firstname`" with the following dtypes:

```python
gender        category
firstname     object
count         int64
year          float64
```

In [20]:
# your code here
#to delete columns
data = data.drop("sexe", axis=1)
#to rename columns 
data = data.rename(columns={'annais':'year','nombre':'count','preusuel':'firstname'})
data.sample(15)

,firstname,year,count,genre
264628,SOSTHENE,2014.0,3,Male
349346,BÉNÉDICTE,2014.0,19,Female
294710,YANN,2013.0,504,Male
329059,ANNA-LIVIA,2019.0,6,Female
515942,MAGUELONNE,1999.0,9,Female
29081,ARONA,2000.0,3,Male
4264,ABDERRAHIM,1970.0,4,Male
575474,NICOLAS,1947.0,6,Female
85979,EVANS,2016.0,27,Male
539250,MARIELLE,2018.0,12,Female


**QUESTION:** How many missing values (NA) have been introduced? How many births are concerned?

In [21]:
# your code here
data.isnull().sum().sum()#number of missing values

data.isnull().any(axis=1).sum() #number of rows containing missing values 

36676

In [22]:
(
print
 (data.isnull()
  .sum()
  .sum()
  ,"missing values have been introduced."
  ,data.isnull().any(axis=1).sum()
  , "births are concerned")
)

36677 missing values have been introduced. 36676 births are concerned


**QUESTION:** Read the documentation and describe the origin of rows containing the missing values.

your code here


According to the documentation. The missing values are due to births where the name has been atrributed at least 20 times 
between 1945 and 2020 but haven't been attributed more than 3 times that year. In those cases, the birth year is replaced by 
XXXX. Indeed, looking at the data, we noticed that the majority of missing values were in the year column

## Checkpointing: save your transformed dataframe

**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2020_csv.zip`. Try several compression methods.

In [23]:
# your code here
try: 
    data.to_csv("nat2020.zip", 
           index=False, 
           compression="zip") #could also be gzip, bz2, xz or None
except:
    print("File already exists")

/usr/lib/python3.8/zipfile.py:1517: UserWarning: Duplicate name: 'nat2020.zip'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2020.hdf` using `.hdf` format

In [25]:
# your code here
import tables as tb
data.to_hdf("./nat2020.hdf", key = 'data',complevel = 2, format = 'table')

At that point your working directory should look like:

```
├── homework01.py      # if you use `jupytext`
├── homework01.ipynb
├── nat2020.hdf
├── nat2020.zip
```

**QUESTION:** Reload the data using `read_hdf(...)` so that the resulting dataframe  is properly typed with meaningful column names.

__Hint:__ use `try: ... except` to handle exceptions such as `FileNotFoundError`

In [26]:
# your code here

data = pd.read_hdf("nat2020.hdf")
data.head(10)#petit bémol, les modifications à data n'ont pas été pris en compte

,firstname,year,count,genre
0,_PRENOMS_RARES,1900.0,1250,Male
1,_PRENOMS_RARES,1901.0,1342,Male
2,_PRENOMS_RARES,1902.0,1330,Male
3,_PRENOMS_RARES,1903.0,1286,Male
4,_PRENOMS_RARES,1904.0,1430,Male
5,_PRENOMS_RARES,1905.0,1472,Male
6,_PRENOMS_RARES,1906.0,1451,Male
7,_PRENOMS_RARES,1907.0,1514,Male
8,_PRENOMS_RARES,1908.0,1509,Male
9,_PRENOMS_RARES,1909.0,1526,Male


## Some data "analytics" and visualization

**QUESTION**: For each year, compute the total number of Female and Male births and the proportion of Female  births among total births

__Hints:__

- Groupby operations using several columns for the groups return a dataframe with a `MultiIndex` index see [Pandas advanced](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)

- Have a look at `MultiIndex`, `reset_index`, `pivot`, `columns.droplevel`

In [28]:
# your code here

total_by_year = pd.pivot_table(data, index = 'year',columns='genre', values = 'count', aggfunc='sum')
total_by_year.sample(10)

genre,Male,Female
year,,
1918.0,169975,178545
1920.0,318031,330290
1938.0,282899,282628
1917.0,148323,158036
2015.0,397664,378390
1947.0,458363,434493
1919.0,188588,196925
1951.0,434956,414722
2005.0,407042,387311


**QUESTION:** Plot the proportion of female births as a function of year. Don't forget: title, axes labels, ticks, scales, etc.

Because of what we did before, the `plot` method of a `DataFrame` with be rendered using `plotly`, so you can use this. But you can use also `seaborn` or any other available plotting library that you want.

__Hint:__ Mind the missing values in the `year` column

In [ ]:
# your code here


**QUESTION:** Make any sensible comment about this plot.

_Insert your answer here_

> ...

**QUESTION:** Explore the fluctuations of sex ratio around its mean value since 1945.
Plot deviations of sex ratio around its mean since 1945 as a function of time.

In [ ]:
# your code here


# Popular firstnames

## The top-20 most popular firstnames since 1900

**QUESTION:** For each firstname and sex (some names may be given to girls and boys), compute the total number of times this firstname has been given during `1900-2019`. Print the top 20 firstnames given and style your result dataframe using `background_gradient` for instance.

In [ ]:
# your code here


## Rare firstnames

**QUESTION:** For each sex, plot the proportion of births given `_PRENOMS_RARES` as a function of the year.

In [ ]:
# your code here


# A study of the "Marie" firstname

**QUESTION:** Plot the proportion of female births given name `'MARIE'` or `'MARIE-...'` (compounded names) as a function of the year.
Proceed in such a way that the reader can see the share of compounded names. We are expecting an _area plot_.

__Hints:__

- Have a look at the `.str` accessor (to apply a string method over a whole column containing string)
- Have a look at [r-graph-gallery: stacked area](https://www.r-graph-gallery.com/stacked-area-graph.html)  and
at [ggplot documentation](https://ggplot2.tidyverse.org/reference/geom_ribbon.html). Pay attention on the way you stack the area corresponding to names matching pattern 'MARIE-.*' over or under the are corresponding to babies named 'MARIE'
- See Graphique 3, page 48, de _L'archipel français_  de J. Fourquet. Le Seuil. Essais. Vol. 898.

- Add annotation, 1st World War, Front Populaire, 2nd World War, 1968

In [ ]:
# your code here


# Top 10 firstnames of year 2000

**QUESTION:** For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.

__Hint:__ Leave aside the rows with '_PRENOMS_RARES'.

In [ ]:
# your code here


# Getting help

- [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/)

- [plotly](https://plotly.com/python/) for animated plots

- [stackoverflow](https://stackoverflow.com)

- [stackoverflow: pandas](https://stackoverflow.com/questions/tagged/pandas)

- [stackoverflow: plotly+python](https://stackoverflow.com/questions/tagged/plotly+python)

- The US `babynames` analogue of the INSEE file has been a playground for data scientists,
 see [https://github.com/hadley/babynames](https://github.com/hadley/babynames)

- Don't Repeat Yourself (DRY) principle  at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)